Poniższy notatnik jest integralną częścią pracy magisterskiej pt. <br>"Metody usprawniania wyników klasyfikacji w problemie wykrywania bankrutujących przedsiębiorstw". 
<br>
W notatniku zamieszczono kod napisany w języku R wykorzystany do wykonania większości obliczeń zamieszczonych w pracy magisterskiej.<br>

# 1 Preprocessing

## 1.1 Wczytywanie danych i pakietów

In [ ]:
library(dplyr)
library(tidyverse)
library(mice)
library(caret)
library(gbm)
library(VIM)
library(MLmetrics)
library(ROCR)

#### Wczytanie danych, zmiana nazw kolumn, konwersja zmiennej Y z int -> factor

In [ ]:
df = read.csv("bankrut.csv")

In [ ]:
colnames(df) <- c("X1", "X2", "X3", "X4", "X5", "X6", "X7", "X8", "X9", "X10", "X11", "X12", "X13", "X14", "X15", "X16", "X17",
                  "X18", "X19", "X20", "X21", "X22", "X23", "X24", "X25", "X26", "X27", "X28", "X29", "X30", "X31", "X32",
                  "X33", "X34", "X35", "X36", "X37", "X38", "X39", "X40", "X41", "X42", "X43", "X44", "X45", "X46", "X47", 
                  "X48", "X49", "X50", "X51", "X52", "X53", "X54", "X55", "X56", "X57", "X58", "X59", "X60", "X61", "X62", 
                  "X63", "X64", "Y")

In [ ]:
df$Y <- as.factor(df$Y)

## 1.2 Brakujące dane

#### Zliczenie brakujących danych w każdej zmiennej oraz zapis do tabeli

In [ ]:
df_nulls = sapply(df, function(x) sum(is.na(x)))
df_nh = df_nulls[df_nulls>0]

In [ ]:
print(df_nh)

In [ ]:
df_nan = data_frame(Attribute = as.character(names(df_nh)), Nr_of_NaNs=df_nh)

#### Histogram zmiennych  posiadających > 200 NA

In [ ]:
df_nh = df_nulls[df_nulls>200]
df_nan = data_frame(Attribute = as.character(names(df_nh)), Nr_of_NaNs=df_nh)
df_nan$Attribute = factor(df_nan$Attribute, levels = df_nan$Attribute[order(-df_nan$Nr_of_NaNs)])
ggplot(df_nan, aes(Attribute, Nr_of_NaNs)) + theme_bw() + geom_bar(stat = "identity")

*Opcjonalnie: Zliczanie wierszy, w których występują NA oraz ich usunięcie*

In [ ]:
row_na <- apply(df, 1, function(x){any(is.na(x))})
sum(row_na)
df_filtered <- df[!row_na,]

*Opcjonalnie: Tworzenie dataframe, w którym liczba NA w wierszu nie przekracza 5*

In [ ]:
df_filtered_5 <- df[!rowSums(is.na(df)) > 5,]

## 1.3 Wartości odstające (outliers)

Zastosowano 2 różne metody eliminujące wartości odstające.<br>
1) Usunięcie i wstawienie NA, w późniejszym stadium zostaną uzupełnione za pomocą imputacji<br>
2) Usunięcie i wstawienie wartości z 95 lub 5 percentyla.<br>

Z uwagi na specyfikę danych (bardzo duża wariancja występująca w większości zmiennych) użyto szerokiego marginesu wykrywania -
10IQR. Standardowa miara 1,5IQR zaklasyfikowałaby zbyt dużo wartości jak outlier. <br>
Za wartości odstające uznane zostały: x < Q1 - 10 x IQR lub x > Q3 + 10 x IQR <br>

#### Wariancja każdej ze zmiennych (z wyłączeniem Y)

In [ ]:
var = apply(df[-65], 2, var, na.rm = TRUE)
var

#### Wyświetlanie wariancji dla kolumn, gdzie var > 100

In [ ]:
round(var[var>100], digits=0)

#### Przygotowanie zmiennych oraz tabeli zliczających wartości odstające

In [ ]:
df_out1 = df
df_out2 = df

count_outliers <- data.frame(Attribute=character(), 
                          Nr_of_outlier=double(),
                          stringsAsFactors = FALSE)

count_outliers2 <- data.frame(Attribute=character(), 
                          Nr_of_outlier=double(),
                          stringsAsFactors = FALSE)

#### Metoda 1 - wstawianie za outlier -> NA

In [ ]:
for (i in which(sapply(df, is.numeric))) {
  
  q = quantile(df[i], na.rm=TRUE)
  iqr = IQR(as.numeric(unlist(df[i])), na.rm=TRUE)
  up_limit = as.numeric(q["75%"]) + (10*iqr)
  down_limit =  as.numeric(q["25%"]) - (10*iqr)
  limits = c(up_limit, down_limit)
  sum_out = sum(df[i]<down_limit | df[i]>up_limit, na.rm=TRUE)
  count_outliers[nrow(count_outliers) + 1,]= list(as.character(names(df[i])), sum_out)
  
  for (j in which(df[i]<down_limit | df[i]>up_limit)) {
    df_out1[j, i] <- NA
  }
}

#### Suma znalezionych outliers

In [ ]:
sum(count_outliers$Nr_of_outlier)
count_outliers

#### Metoda 2 - wstawianie za outlier -> 5th albo 95th percentile

In [ ]:
for (i in which(sapply(df, is.numeric))) {
  
  qnt <- quantile(df[i], probs=c(.25, .75), na.rm = T)
  outer_qnt <- quantile(df[i], probs=c(.05, .95), na.rm = T)
  H <- 10 * IQR(as.numeric(unlist(df[i])), na.rm = T)
  up_limit = qnt[2] + H
  down_limit =  qnt[1] - H
  sum_out = sum(df[i]<down_limit | df[i]>up_limit, na.rm=TRUE)
  results_outliers2[nrow(results_outliers2) + 1,]= list(as.character(names(df[i])), sum_out)
  
  for (j in which(df[i]<down_limit | df[i]>up_limit)) {
    df_out2[j, i][df_out2[j, i] < (qnt[1] - H)] <- outer_qnt[1]
    df_out2[j, i][df_out2[j, i] > (qnt[2] + H)] <- outer_qnt[2]
  }
}
sum(results_outliers2$Nr_of_outlier)

## 1.4 Imputacja

Imputacji poddano 3 zbiory otrzymane we wcześniejszych punktach:<br>
**df** - oryginalny zbiór zawierający outliers<br>
**df_out1** - zbiór z usuniętymi outliers i wstawionymi NA<br>
**df_out2** - zbiór z usuniętymi outliers i wstawionymi wartościami z 5 lub 95 percentyla<br><br>
Na każdym z nich wykonany 5 różnych metod imputacji:<br>
1) Uzupełnienie medianą (obliczona osobno dla każdej z klas)<br>
2) Algorytm k-NN<br>
3) Hot-Deck<br>
4) Imputacja wielokrotna<br>
5) Imputacja wielokrotna uśredniona<br><br>
Poniżej przedstawiono imputację dla zbioru **df**

#### 1) Mediana warunkowa

In [ ]:
df_med = df
if_row_nan = df_med[, "Y"] == df_med[j, "Y"]
for (i in which(sapply(df_med, is.numeric))) {
  for (j in which(is.na(df_med[, i]))) {
    df_med[j, i] <- median(df_med[if_row_nan, i],  na.rm = TRUE)
  }
}

#### 2) Algorytm k-NN

In [ ]:
df_k = preProcess(df, method = c("knnImpute"), k=3)
df_knn = predict(df_k, df)

#### 3) Hot-Deck

In [ ]:
df_hot = hotdeck(df, imp_var = FALSE)

#### 4) Imputacja wielokrotna

In [ ]:
imp <- mice(df, m = 5, maxit = 10,  seed = 8, method = 'pmm')
df_mice = complete(imp)

#### 5) Imputacja wielokrotna uśredniona

In [ ]:
df_mice_mean = df

for(i in 1:5){
  assign(paste("df_mice", i, sep = ""), complete(imp, i))    
}

for (i in which(sapply(df, is.numeric))) {
  for (j in which(is.na(df_mice_mean[, i]))) {
    df_mice_mean[j, i] <- ((df_mice1[j, i] + df_mice2[j, i] + 
                            df_mice3[j, i] + df_mice4[j, i] + df_mice5[j, i])/5)
  }
}

## 1.5 Skalowanie 

#### Metoda standaryzacji - sprowadzenie zmiennych do rozkładu (0,1)

In [ ]:
#df_med
df_s = preProcess(df_med, method = c("center", "scale"))
df_med = predict(df_s, df_med)

#df_knn
df_k = preProcess(df_knn, method = c("center", "scale"))
df_knn = predict(df_k, df_knn)

#df_hot
df_h = preProcess(df_hot, method = c("center", "scale"))
df_hot = predict(df_s, df_hot)

#df_mice
df_m = preProcess(df_mice, method = c("center", "scale"))
df_mice = predict(df_m, df_mice)

#df_mice_mean
df_mm = preProcess(df_mice_mean, method = c("center", "scale"))
df_mice_mean = predict(df_mm, df_mice_mean)

## 1.6 Przygotowanie zbiorów
Ze względu na późniejsze wykonywanie klasyfikacji na 15 różnych zbiorach danych, zbiory te umieszczono dla ułatwienia w jednej liście elementów.

In [ ]:
df_list = list(df_med, df_knn, df_hot, df_mice, df_mice_mean,
               df_out1_med, df_out1_knn, df_out1_hot, df_out1_mice, df_out1_mice_mean,
               df_out2_med, df_out2_knn, df_out2_hot, df_out2_mice, df_out2_mice_mean)

Podobnie postąpiono z nazwami zmiennych, metodami imputacji oraz usuwania outlierów. Ułatwi to wprowadzanie do tabeli wyników informacji na temat specyfiki danego zbioru.

In [ ]:
df_list_names = c("df_med", "df_knn", "df_hot", "df_mice", "df_mice_mean",
                  "df_out1_med", "df_out1_knn", "df_out1_hot", "df_out1_mice", "df_out1_mice_mean",
                  "df_out2_med", "df_out2_knn", "df_out2_hot", "df_out2_mice", "df_out2_mice_mean",
                  "df", "df_out1", "df_out2")

df_list_imputation = c("mediana", "knn", "hotdeck", "mice", "mice_mean",
                       "mediana", "knn", "hotdeck", "mice", "mice_mean",
                       "mediana", "knn", "hotdeck", "mice", "mice_mean",
                       "brak", "brak", "brak")

df_list_outliers = c("brak", "brak", "brak", "brak", "brak",
                    "nan", "nan", "nan", "nan", "nan",
                    "5_95", "5_95", "5_95", "5_95", "5_95", 
                    "brak", "nan", "5_95")

## 1.7 Analiza imputacji na podstawie zmiennych X37 oraz X21
Zmienne te posiadają najwięcej wartości pustych, dlatego też warto poddać analizie jej rozkład przed i po zastosowaniu imputacji<br>
Poniżej kod dla zmiennej X37

#### Tabele zapisujące średnie i odchylenia standardowe

In [ ]:
results_x37 <- data.frame(Metoda = character(),
                          Imputacja = character(),
                          Outliers = character(),
                          mean_0 = double(),
                          mean_all = double(),
                          mean_1 = double(),
                          sd_0 = double(),
                          sd_all = double(),
                          sd_1 = double(),
                          stringsAsFactors = FALSE)

#### Funkcja do_estim_X37() 
Oblicza średnie i odchylenia standardowe zmiennej X37 dla każdej klasy

In [ ]:
do_estim_X37 = function(df_input, dfname, imputation, outliers){
  # Z wybranego zbioru zwraca średnią i odchylenie zmiennej X37 dla kazdej klasy
  mean_0 = mean(df_input$X37[df_input$Y == 0], na.rm = TRUE)
  mean_all = mean(df_input$X37, na.rm = TRUE)
  mean_1 = mean(df_input$X37[df_input$Y == 1], na.rm = TRUE)
  sd_0 = sd(df_input$X37[df_input$Y == 0], na.rm = TRUE)
  sd_all = sd(df_input$X37, na.rm = TRUE)
  sd_1 = sd(df_input$X37[df_input$Y == 1], na.rm = TRUE)
  means_X37 = list(dfname, imputation, outliers, mean_0, mean_all, mean_1, sd_0, sd_all, sd_1)
  means_X37
}

Pętla obliczająca statystyki dla każdego z 18 zbiorów

In [ ]:
for (i in 1:18) {
  results_x37[nrow(results_x37) + 1,] = do_estim_X37(df_list[[i]], df_list_names[i], 
                                                     df_list_imputation[i], df_list_outliers[i])
}

# 2 Tabele i funkcje

Zapisywanie otrzymanych wyników w przejrzysty i praktyczny (pozwalający na wykorzystanie w dalszej analizie sposób wymaga stworzenia odpowiednych tabel typu dataframe. Ponadto niezbędne stały się funkcje umożliwiające korzystanie z niestandardowych miar ocen jakości klasyfikacji.

## 2.1 Zapisywanie wyników klasyfikacji

#### Tabela zbierająca wyniki klasyfikacji

In [ ]:
results <- data.frame(Metoda = character(),
                      Imputacja = character(),
                      Outliers = character(),
                      AUC_train_MN = double(),
                      AUC_train_SD = double(),
                      AUC = double(),
                      Recall_train_MN = double(),
                      Recall_train_SD = double(),
                      Recall = double(),
                      F1 = double(),
                      Kappa = double(),
                      Zbiór = character(),
                      Trening = character(),
                      Parametry = character(),
                      stringsAsFactors = FALSE)

#### Funkcja dodajWynik()
Odpowiada za zapisywanie do tabeli **results** wszystkich niezbędnych w dalszej analizie parametrów klasyfikacji

In [ ]:
dodajWynik <- function(input_frame, impute_method, outlier_method, model, modelControl, matCon){
  
  cvString <- paste(modelControl$number,modelControl$method, sep="-")
  
  bT = model$bestTune
  col_bT = colnames(bT)
  c = list()
  for (i in 1:ncol(model$bestTune)){
    c[i] = paste(col_bT[i], bT[i], sep=": ")
  }
    
  tuneString <- paste(c[1], c[2], c[3], c[4], sep = " / ")

  results[nrow(results) + 1,] <<- list(model$method,
                                    impute_method,
                                    outlier_method,
                                    round(mean(model$resample$ROC), digits = 4),
                                    round(sd(model$resample$ROC), digits = 4),
                                    round(auc_test, digits = 4),
                                    round(mean(model$resample$Recall), digits = 4),
                                    round(sd(model$resample$Recall), digits = 4),
                                    round(matCon$byClass['Recall'], digits = 4), 
                                    round(matCon$byClass['F1'], digits = 4), 
                                    round(matCon$overall['Kappa'], digits = 4),
                                    input_frame, 
                                    cvString,
                                    tuneString)
  print("Dodano wynik do tabeli")
}

## 2.2  Miary oceny klasyfikacji

#### Funkcja metricSummary()
Modyfikacja domyślnej funkcji **twoClassSummary()**, która znajduje się w pakiecie **caret**. Zmianie w stosunku do pierwotnej funkcji uległa kolejność pobierania z danych klasy pozytywnej (1) i negatywnej(0) oraz dodano inne miary klasyfikacji stosowane w uczeniu klasyfikatora.

In [ ]:
library(MLmetrics)

metricSummary <- function(data, lev = NULL, model = NULL){
  
  lvls <- levels(data$obs)
  
  if (length(lvls) > 2) 
    stop(paste("Your outcome has", length(lvls), "levels. The twoClassSummary() function isn't appropriate."))
  
  caret:::requireNamespaceQuietStop("ModelMetrics")
  
  if (!all(levels(data[, "pred"]) == lvls)) 
    stop("levels of observed and predicted data do not match")
  
  data$y = as.numeric(data$obs == lvls[2])
  rocAUC <- ModelMetrics::auc(ifelse(data$obs == lvls[1], 0, 1), data[, lvls[2]])
    
  out <- c(rocAUC, 
           sensitivity(data[, "pred"], data[, "obs"], lvls[2]), 
           specificity(data[, "pred"], data[, "obs"], lvls[1]),
           recall(data[, "pred"], data[, "obs"], lvls[2])
           F1_Score(y_pred = data[, "pred"], y_true = data[, "obs"], positive = lvls[2]))
  )
    
  names(out) <- c("ROC", "Sens", "Spec", "Recall", "F1")
  out
  
}

#### Funkcja oblicz_auc()
Funkcja obliczająca miarę **AUC** dla danych testowych

In [ ]:
oblicz_auc <- function(y_predicted_probs, df_labels){
  
  roc_pred = prediction(predictions = y_predicted_probs$X1, labels = df_labels$Y)
  perf = performance(roc_pred, measure = 'tpr', x.measure = 'fpr')
  perf.auc = performance(roc_pred, measure = "auc")
  auc_val = unlist(perf.auc@y.values)
  
  auc_val
}

# 3 Klasyfikacja

#### Omówienie schematu algorytmu na przykładzie klasyfikatora gbm oraz zbioru df_med

Wczytanie danych, podział na zbiory testowe i treningowe oraz podział na zmienne wejściowe (X1 - X64) i zmienną wyjściową (Y)

In [ ]:
df_e = df_med

trainIndex = createDataPartition(df_e$Y, p=0.8, list=FALSE)
df_train = df_e[trainIndex,]
df_test = df_e[-trainIndex,]
X_train = df_train[,1:ncol(df_e)-1]
y_train = df_train$Y
X_test = df_test[,1:ncol(df_e)-1]
y_test = df_test$Y

levels(df_train$Y) <- c('X0', 'X1')
levels(df_test$Y) <- c('X0', 'X1')

Funkcja trainControl() pozwala na kontrolowanie treningu oraz zastosowanie metod próbkowania danych w procesie uczenia

In [ ]:
gbmControl = trainControl(method = "cv",
                          number = 5,
                          search = "grid",
                          classProbs = TRUE,
                          summaryFunction = metricSummary)

Funkcja expan.grid() daje możliwość sprecyzowania hiperparametrów wprowadzanych do modelu

In [ ]:
gbmGrid = expand.grid(interaction.depth = 9,
                      n.trees = 500,
                      shrinkage = 0.10,
                      n.minobsinnode = 20)

Funkcja train() służy do treningu danego klasyfikatora. Przekazane zostają do niej wcześniej wspomniane funkce trainControl() oraz expand.grid()

In [ ]:
gbm = train(Y ~ ., data = df_train, 
                      method = "gbm",
                      trControl = gbmControl,
                      tuneGrid = gbmGrid,
                      metric = 'Recall',
                      na.action = na.pass)

Wykonanie predykcji dla danych testowych oraz obliczenie prawdopodobieństw potrzebnych do obliczenia miary AUC

In [ ]:
y_pred = predict(gbm, df_test, na.action = na.pass)
y_pred_probs = predict(gbm, df_test, type="prob", na.action = na.pass)

Stworzenie macierzy pomyłek na podstawie predykcji dla danych testowych oraz rzeczywistych danych

In [ ]:
conf_test = confusionMatrix(reference = df_test$Y, data = y_pred, mode = 'everything', positive = 'X1')

Uśredniona macierz pomyłek dla danych treningowych

In [ ]:
conf_train_cv = confusionMatrix(gbm, "average")

Obliczenie miary AUC

In [ ]:
auc_test = oblicz_auc(y_pred_probs, df_test)

In [ ]:
print(conf_train_cv)
print(conf_test)

Zapisywanie wyników klasyfikacji do tabeli **results**

In [ ]:
dodajWynik(df_med, mediana, gbm, gbmControl, conf_test)